File log.csv contains history of user's activity.
Ich row is one event described as "time", "user id", "event type", "evennt's paramter".

1. Count total number of user's sessions in 14 April 2020.
2. Find a day when number of unique users listening music is highest.
3. Find 15 minutes interval when number of user's events is highest.

In [1]:
import pandas as pd
from datetime import timedelta
from datetime import datetime
import seaborn as sns

In [2]:
def count_sessions(log):
    last_event_time = log.iloc[0]
    sessions = 1
    for event in log[1:]:
        delta = (event - last_event_time).total_seconds() / 60
        if delta > 30:
            sessions += 1
        last_event_time = event
    return sessions

def count_all_sessions():
    data = pd.read_csv("log.csv")
    data["date"] = data["date"].str.replace("_", " ")
    data["date"] = pd.to_datetime(data["date"])
    users_activity = {}

    start = pd.to_datetime("2020-04-14 00:00:00")
    stop = pd.to_datetime("2020-04-15 00:00:00")
    data_1 = data[(data["date"] >= start) & (data["date"] < stop)]

    for user in data_1["user"].values:
        users_activity[user] = count_sessions(data_1[data_1["user"] == user]["date"])

    activity = pd.DataFrame().from_dict(users_activity, orient="index", columns=["sessions"])
    return activity.sessions.sum()

def find_music_day():
    data = pd.read_csv("log.csv")
    data["date"] = data["date"].str.replace("_", " ")
    data["date"] = pd.to_datetime(data["date"])
    data_2 = data[(data["parameter"] == 'music') & (data["event_type"] == 1)]
    
    users_per_day = {}
    for day in range(1, 31):
        start = pd.to_datetime(f"2020-04-{day} 00:00:00")
        stop = pd.to_datetime(f"2020-04-{day} 23:59:59")
        users_per_day[f"2020-04-{day}"] = len(data_2[(data_2["date"] >= start) & (data_2["date"] <= stop)].user.unique())
    
    return [max(users_per_day, key=users_per_day.get), users_per_day[max(users_per_day, key=users_per_day.get)]]

def find_interval():
    data = pd.read_csv("log.csv")
    data["date"] = data["date"].str.replace("_", " ")
    data["date"] = pd.to_datetime(data["date"])

    start = pd.to_datetime("2020-04-01 00:00:00")
    stop = pd.to_datetime("2020-04-30 23:59:59")
    max_events = 0
    max_time = 0
    time = start
    while (time + timedelta(minutes=15)) <= stop:
        current = len(data[(data["date"] >= time) & (data["date"] <= time + timedelta(minutes=15))])
        if current > max_events:
            max_events = current
            max_time = time
        time = time + timedelta(seconds=1)
    return [max_time, max_events]

In [3]:
count_all_sessions()

4103

In [4]:
find_music_day()

['2020-04-22', 378]

In [5]:
find_interval()

[Timestamp('2020-04-07 12:34:16'), 179]